# Trabalho Final - Aprendizado por Reforço

### Importando os pacotes

In [176]:
import numpy as np
import pandas as pd
import random
from gym import Env
from gym.spaces import Discrete, Box
from sklearn.model_selection import train_test_split
import time

import tensorflow as tf

from stable_baselines3.dqn import MlpPolicy, MultiInputPolicy
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common import monitor 
from stable_baselines3.common import logger
from stable_baselines3.common.evaluation import evaluate_policy

### Separando os Conjuntos Teste x Treino

In [182]:
# Importando os dados do Excel
df_dataset = pd.read_excel('./DadosGolfitSoccer_2024_07_03.xlsx')

# Removendo colunas desnecessárias
df_dataset.drop(columns=['Codigo', 'Ano_Avaliacao', 'Nome', 'CategoriaEtaria', 'Ano_Nascimento','CLUBE', 'PosicaoJogo'], inplace=True)

# Removendo as linhas com valores vazios
df_dataset.dropna(
    axis=0,
    how='any',
    subset=None,
    inplace=True
)

# Separando o conjunto de features
X = df_dataset.drop('Sucesso', axis=1)
# Separando o conjunto de rótulos
y = df_dataset['Sucesso']

# Separando os conjuntos de teste e treino
# O parâmetro de estratificação é utilizado para balancear a quantidade das classes entre todas as divisões
# O parâmetro random_state é utilizado como seed para a divisão dos conjuntos de teste e treino
df_train_X, df_test_X, df_train_y, df_test_y = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)


# Retirando a coluna da classe - Sucesso
df_train_X = np.array(df_train_X)
# Convertendo a coluna para o tipo que será usado pelo modelo
df_train_y = np.array(df_train_y)

# Dataset - Teste
# Retirando a coluna da classe - Sucesso
df_test_X = np.array(df_test_X)
# Convertendo a coluna para o tipo que será usado pelo modelo
df_test_y = np.array(df_test_y)

### Calculando a taxa de desbalanceamento entre as classes
![taxa_desbalanceamento](./taxa_desbalanceamento.png)

|DP|/|DN|

In [183]:
# Calculando a quantidade de instâncias pertencentes à classe majoritária - DN
quantity_DN = (df_train_y == 0).sum()
# Calculando a quantidade de instâncias pertencentes à classe minoritária - DP
quantity_DP = (df_train_y == 1).sum()
# Calculando a taxa de desbalanceamento do DataSet
calculated_imbalance_rate = quantity_DP/quantity_DN

print("DATASET TREINO")
print("Quantidade de atletas da classe majoritária: ", quantity_DN)
print("Quantidade de atletas da classe minoritária: ", quantity_DP)
print("Taxa de desbalanceamento do DataSet: ", calculated_imbalance_rate)

# Calculando a quantidade de instâncias pertencentes à classe majoritária - DN
quantity_DN = (df_test_y == 0).sum()
# Calculando a quantidade de instâncias pertencentes à classe minoritária - DP
quantity_DP = (df_test_y == 1).sum()
# Calculando a taxa de desbalanceamento do DataSet
calculated_imbalance_rate = quantity_DP/quantity_DN

print("\nDATASET TESTE")
print("Quantidade de atletas da classe majoritária: ", quantity_DN)
print("Quantidade de atletas da classe minoritária: ", quantity_DP)
print("Taxa de desbalanceamento do DataSet: ", calculated_imbalance_rate)

DATASET TREINO
Quantidade de atletas da classe majoritária:  63
Quantidade de atletas da classe minoritária:  14
Taxa de desbalanceamento do DataSet:  0.2222222222222222

DATASET TESTE
Quantidade de atletas da classe majoritária:  27
Quantidade de atletas da classe minoritária:  6
Taxa de desbalanceamento do DataSet:  0.2222222222222222


### Criando o ambiente para classificação

In [173]:
# Env: uma classe python de alto-nível que representa um MDP(Processo de Decisão de Markov)
#      permite ao usuário gerar o estado inicial, transição de estados e visualização do ambiente
# Instanciando um ambiente a partir do ambiente fornecido pelo Gym
class PlayersEnv(Env):
    # def __init__(self, majority_class=0, minority_class=1, row_per_episode=1, dataset=(df_train_X, df_train_y), imbalance_rate=calculated_imbalance_rate):
    def __init__(self, majority_class, minority_class, row_per_episode, dataset, imbalance_rate):
        # As ações que o agente podem tomar são discretas, envolvendo classificar entre 2 classes
        self.action_space = Discrete(2)
        
        # O espaço de observações consiste nos jogadores que o agente deve classificar - 42 features para cada jogador
        # obersevation = np.array([[np.finfo('float32').max] * 42], dtype=np.float32)
        self.observation_space = Box(0, 60, shape=(42,), dtype=np.float32)
        # }, seed=42)
        # self.observation_space = Box(-obersevation, obersevation, shape=(1, 42), dtype=np.float32)

        # Definindo qual as classe majoritária/minoritária do dataset - Para lidar com desbalanceamento dos dados
        self.majority_class = majority_class
        self.minority_class = minority_class

        # Definindo a taxa de desbalanceamento entre as classes majoritária e minoritária
        self.imbalance_rate = imbalance_rate

        # self.observations_space = Box(low=0, high=1, shape=())
        # Estado inicial em que o agente começa a interação
        self.state = 0
        # Tamanho do episódio == quantidade de instâncias (jogadores) a serem classificados
        self.episode_length = 0

        ## Armazenamento de métricas
        # Verdadeiro Positivo
        self.TP = 0
        # Falso Positivo
        self.FP = 0
        # Verdadeiro Negativo
        self.TN = 0
        # Falso Negativo
        self.FN = 0
        

        # Definição de parâmetros
        self.row_per_episode = row_per_episode
        self.step_count = 0
        self.x, self.y = dataset
        self.dataset_idx = 0
        

    # Função que especifica o que o agente fará a cada passo - Representa uma ação do agente
    # Recebe a ação tomada pelo agente como parâmetro
    def step(self, action):
        # Verificar se o agente classificou certo
        # Diferenciar recompensas para a classe maioritária e minoritária
        # Utilizar a fórmula definida pelos autores na metodologia do artigo

        # Flag que define se a tomada de decisão está finalizada
        done = False
    
        # Definição das recompensas caso o agente tome a decisão CORRETA
        if action == self.expected_action:
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    # Retorna a recompensa pela ação
                    reward = self.imbalance_rate * 1
                    # Incrementa os Verdadeiros Negativos (TN)
                    self.TN += 1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    # Retorna a recompensa pela ação
                    reward = 1
                    # Incrementa os Verdadeiros Positivos (TP)
                    self.TP += 1

        # Definição das recompensas caso o agente tome a decisão INCORRETA
        if action != self.expected_action:
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    # Retorna a recompensa pela ação
                    reward = self.imbalance_rate * -1
                    # Incrementa os Falso Negativos (FN)
                    self.FP += 1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    # Retorna a recompensa pela ação
                    reward = -1
                    # Incrementa os Verdadeiros Negativos (TN)
                    self.FN += 1

        # Passa para a próxima observação do dataset
        observation = self._next_observation()

        # Aumenta a contagem de decisões tomadas
        self.step_count += 1

        # Caso a quantidade de passos definidos por episódio tenha sido alcançada
        if self.step_count >= self.row_per_episode:
            done = True

        return observation, reward, done, {}  

    # Função que reseta o ambiente para treinar novamente
    def reset(self):
        # Zera a contagem de passos
        self.step_count = 0
        # Define a nova observação inicial
        observation = self._next_observation()
        # Retorna a observação inicial
        return observation

    # Subrotina que retorna a próxima observação para o agente
    def _next_observation(self):
        # Define um índice de forma aleatória para a próxima observação
        next_observation_idx = random.randint(0, len(self.x) - 1)
        # Define a ação esperada para a observação - Retorna o rótulo da observação
        self.expected_action = int(self.y[next_observation_idx])
        # Extrai a observação com o índice aleatório definido anteriormente
        observation = self.x[next_observation_idx]

        return observation

    # Subrotina que calcula as métricas do modelo
    def metrics(self):
        TPrate = self.TP / (self.TP + self.FN) 
        TNrate = self.TN / (self.TN + self.FP)
        FPrate = self.FP / (self.TN + self.FP)
        FNrate = self.FN / (self.TP + self.FN)
        PPvalue = self.TP / (self.TP + self.FP)
        NPvalue = self.TN / (self.TN + self.FN)

        G_mean = np.sqrt(TPrate * TNrate)

        Recall = TPrate = self.TP / (self.TP + self.FN)
        Precision = PPvalue = self.TP / (self.TP + self.FP)
        F_measure = 2 * Recall * Precision / (Recall + Precision)
 
        return TPrate, TNrate, G_mean, F_measure

### Treinamento do modelo

In [184]:
# Instanciando o ambiente para treinamento
environment = PlayersEnv(
    majority_class=0, 
    minority_class=1, 
    row_per_episode=1, 
    dataset=(df_train_X, df_train_y), 
    imbalance_rate=calculated_imbalance_rate
)

environment.observation_space.shape

# Inicializando o modelo DQN
model = DQN(
    "MlpPolicy", 
    DummyVecEnv([lambda: environment]), 
    learning_rate=0.001, 
    gamma=0.99
)

# Treinando o modelo
model.learn(total_timesteps=77)

# Salvando o modelo para validação
model.save("IC_MDP_DQN")

C:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
# No modelo DQNIMB In DQNimb model, a recompensa para a classe minoritária é 1 e para a clase majoritária é λ
# Testar diferentes valores de lambda λ
# {0.05ρ, 0.1ρ, 0.5ρ, ρ, 5ρ, 10ρ, 20ρ}.

### Validando o modelo

In [185]:

# Instanciando o ambiente para teste
test_environment = PlayersEnv(
    majority_class=0, 
    minority_class=1, 
    row_per_episode=1, 
    dataset=(df_test_X, df_test_y), 
    imbalance_rate=calculated_imbalance_rate
)

# Carregando o modelo treinado anteriormente
model = DQN.load("IC_MDP_DQN", env=DummyVecEnv([lambda: test_environment]))

# Define a observação inicial
test_environment = model.get_env()
observation = test_environment.reset()

# Percorre o DataSet de teste
for i in range(33):
    # Faz uma previsão com o modelo
    action, done = model.predict(observation)
    # Retorna a próxima observação
    observation = test_environment.reset()


# Imprimindo as métricas do modelo
sensitivity, specificity, g_mean_metric, f_measure_metric = environment.metrics()

# print("Sensibilidade (Recall/ Taxa de verdadeiros positivos): ", TPrate)
# print("Especificidade (Taxa de verdadeiros negativos): ", TNrate)
# print("Sensibilidade (Recall): ", TPrate)
# print("Sensibilidade (Recall): ", TPrate)
# print("Sensibilidade (Recall): ", TPrate)
# print("Sensibilidade (Recall): ", TPrate)

sensitivity, specificity, g_mean_metric, f_measure_metric

(0.4666666666666667, 0.47692307692307695, 0.47176700029156615, 0.25)